In [57]:
import pandas as pd
result_filename = "assignments_v1.csv"
hits_filename = "cache/hits_v1.txt"
out_filename = "data_analysis_v1.csv"

file = open(result_filename, "r")
df = pd.read_csv(file)
df = df.reset_index()

In [58]:
# clean string, lower, stem
# import nltk
# from nltk.stem.porter import PorterStemmer
# stemmer = PorterStemmer()

def preprocess(raw_str):
    clean_str = ''.join([letter for letter in raw_str.lower().replace('+',' ') if letter not in punc])
    clean_str_without_stopwords = ' '.join(set([word for word in clean_str.split() if word not in stopwords.words()]))
    return nlp(clean_str_without_stopwords)

import spacy
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nlp = spacy.load('en_core_web_md')
df = df.rename(columns={"Answer": "RawAnswer"})
punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
df["CleanAnswer"] = pd.Series(dtype='object')
for index, row in df.iterrows():
    df.at[index,"CleanAnswer"] = preprocess(row["RawAnswer"])


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/henrywu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [59]:
# add ground truth
reference = {}
with open(hits_filename, "r") as hits_f:
    with open("dev.csv", "r") as ref_f:
        ref_df = pd.read_csv(ref_f)
        for i, hit_id in enumerate(hits_f.readlines()):
            reference[hit_id.strip()] = ref_df.iloc[i]["labels"].strip().replace(";"," ")

df["GroundTruth"] = pd.Series(dtype='object')
for index, row in df.iterrows():
    df.at[index,"GroundTruth"] = preprocess(reference[row["HITId"]])

In [60]:
# average word vector
import numpy as np
df["AnswerWord2VectorAverage"] = pd.Series(dtype='object')
df["GroundTruthWord2VectorAverage"] = pd.Series(dtype='object')
df["similarity"] = pd.Series(dtype='float')
for index, row in df.iterrows():
    df.at[index,"AnswerWord2VectorAverage"] = np.array([token.vector for token in row["CleanAnswer"]]).mean(axis=0)
    df.at[index,"GroundTruthWord2VectorAverage"] = np.array([token.vector for token in row["GroundTruth"]]).mean(axis=0)
    df.at[index,"similarity"] = np.sum(df.iloc[index]["AnswerWord2VectorAverage"]*df.iloc[index]["GroundTruthWord2VectorAverage"])/np.linalg.norm(df.iloc[index]["AnswerWord2VectorAverage"])/np.linalg.norm(df.iloc[index]["GroundTruthWord2VectorAverage"])
    # 300
df

,index,AssignmentId,WorkerId,HITId,AutoApprovalTime,AcceptTime,SubmitTime,RawAnswer,CleanAnswer,GroundTruth,AnswerWord2VectorAverage,GroundTruthWord2VectorAverage,similarity
0,0,384PI804X6L7251BMYLBRV3HJ3HS08,A2JX2VIJWXHS4X,3EGKVCRQG9C2WUI1TI5O4ERL1PNYBW,2023-02-24 13:58:37-05:00,2023-01-25 13:58:04-05:00,2023-01-25 13:58:37-05:00,pop music+drum,"(music, pop, drum)","(rimshot, percussion, kit, drum)","[-0.1119233, 2.6801803, 1.0016, -2.3493998, -1...","[-1.2645626, 0.37058252, -1.0604249, 2.4054751...",0.527812
1,1,3HHRAGRYXMPA07XVFDNV01U9PI99O5,A2MIJ8L80AS9LG,3EGKVCRQG9C2WUI1TI5O4ERL1PNYBW,2023-02-24 14:37:09-05:00,2023-01-25 14:36:44-05:00,2023-01-25 14:37:09-05:00,disco music+drum,"(music, drum, disco)","(rimshot, percussion, kit, drum)","[-0.37319002, 1.2522134, 0.036236662, -1.82216...","[-1.2645626, 0.37058252, -1.0604249, 2.4054751...",0.581920
2,2,3TAYZSBPLZSEYF4P1LPPAJWWG11S2S,A2JX2VIJWXHS4X,3I4E7AFQ3XJCXRHJ74V6ULIJUSETJN,2023-02-24 13:59:37-05:00,2023-01-25 13:58:38-05:00,2023-01-25 13:59:37-05:00,showering,(showering),"(bathtub, pets, gush, water)","[-2.2808, 0.81769, -1.8289, -0.068183, 3.957, ...","[-0.26019, 1.8438125, -3.2307024, -1.0729251, ...",0.523774
3,3,3QL2OFSM9K2UGK4MQIX9N2VC5T6NCK,A2JX2VIJWXHS4X,3NSCTNUR3C7ZBW3PC20VY69MO2Z5AT,2023-02-24 13:56:16-05:00,2023-01-25 13:55:12-05:00,2023-01-25 13:56:16-05:00,woman talk+pet,"(talk, woman)","(pets, caterwaul, speech, meow)","[1.06154, 1.3629, -2.14945, -3.35785, -2.25293...","[0.21570003, 1.10765, -3.2933626, -4.954488, 1...",0.344519
4,4,3ZOTGHDK5WVS5MTUOJNMHWZZH7FOSE,A29A6V8IJE8DQ7,3NSCTNUR3C7ZBW3PC20VY69MO2Z5AT,2023-02-27 14:18:33-05:00,2023-01-28 14:18:03-05:00,2023-01-28 14:18:33-05:00,mom speaking to her baby+baby+mother,"(baby, mom, mother, speaking)","(pets, caterwaul, speech, meow)","[2.0975025, 0.39419246, -3.5506, -2.710025, -0...","[0.21570003, 1.10765, -3.2933626, -4.954488, 1...",0.484349
5,5,3VW6495TLXK1VM95TCO919L2C1SYYT,A2JX2VIJWXHS4X,3N7PQ0KLJI9ZNUXXEOEOFS1TVL63EW,2023-02-24 13:58:04-05:00,2023-01-25 13:57:38-05:00,2023-01-25 13:58:04-05:00,helicopter,(helicopter),"(vehicle, helicopter, aircraft)","[2.2407, 1.9109, -0.51748, -0.70254, -0.82126,...","[1.9864668, 1.8513833, -2.4507601, -0.3573, 1....",0.835747
6,6,36PW28KO4DG2TZUBD5YZRKBFRYLEAL,A2MIJ8L80AS9LG,3N7PQ0KLJI9ZNUXXEOEOFS1TVL63EW,2023-02-24 14:35:58-05:00,2023-01-25 14:33:15-05:00,2023-01-25 14:35:58-05:00,vehicle+construction,"(construction, vehicle)","(vehicle, helicopter, aircraft)","[-0.69105, 0.19879997, -3.29246, -0.2849195, 2...","[1.9864668, 1.8513833, -2.4507601, -0.3573, 1....",0.754394
7,7,39JEC75378L7OPDVS9J0XJPPRXQVCT,A29A6V8IJE8DQ7,3N7PQ0KLJI9ZNUXXEOEOFS1TVL63EW,2023-02-27 14:16:53-05:00,2023-01-28 14:16:02-05:00,2023-01-28 14:16:53-05:00,plane propeller (fan),"(propeller, plane)","(vehicle, helicopter, aircraft)","[-0.69922996, 2.5164251, -2.3631, 0.897875, 1....","[1.9864668, 1.8513833, -2.4507601, -0.3573, 1....",0.758387
8,8,37W3JXSD6KSX6HE5EB6LGC0M1TSWYS,A2JX2VIJWXHS4X,3L55D8AUGNHL6ZA77I1JFYX3JANYC3,2023-02-24 14:00:30-05:00,2023-01-25 13:59:38-05:00,2023-01-25 14:00:30-05:00,speech,(speech),"(insect, bee, fly)","[-1.801, -2.1598, -0.33735, -0.33215, 4.8358, ...","[2.1473234, 0.10906664, -1.9876295, -0.0710733...",-0.009359
9,9,3Z7EFSHGNNYZSEIQSOKIL4MA77GXCE,A2QHKSGU21S0GA,3L55D8AUGNHL6ZA77I1JFYX3JANYC3,2023-02-24 14:05:36-05:00,2023-01-25 14:04:49-05:00,2023-01-25 14:05:36-05:00,man talking+drump,"(talking, drump)","(insect, bee, fly)","[0.18874, -0.28342, -0.765, -1.70425, -0.4649,...","[2.1473234, 0.10906664, -1.9876295, -0.0710733...",0.186652


In [61]:
# average score
df.to_csv(out_filename)
df["similarity"].mean()

0.50931401790953